In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])

Name: /physical_device:GPU:0   Type: GPU


In [4]:
embedding_layer = layers.Embedding(1000, 5)

In [5]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[-0.03762355, -0.03896648, -0.0483936 , -0.03363389,  0.04606378],
       [ 0.00411361,  0.00932074,  0.04543765, -0.02871795,  0.04904887],
       [ 0.00137981, -0.04401372,  0.04046656, -0.00261844, -0.02014222]],
      dtype=float32)

In [6]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

TensorShape([2, 3, 5])

In [7]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to C:\Users\sunlin\tensorflow_datasets\imdb_reviews\subwords8k\0.1.0. Subsequent calls will reuse this data.


In [8]:
encoder = info.features['text'].encoder
encoder.subwords[:20]

['the_',
 ', ',
 '. ',
 'a_',
 'and_',
 'of_',
 'to_',
 's_',
 'is_',
 'br',
 'in_',
 'I_',
 'that_',
 'this_',
 'it_',
 ' /><',
 ' />',
 'was_',
 'The_',
 'as_']

In [9]:
padded_shapes = ([None],())
train_batches = train_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)
test_batches = test_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)

In [10]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()

array([[  62,    9,    4, ...,  937, 2063, 7962],
       [ 597,    1, 2848, ...,    0,    0,    0],
       [  12,  704,  646, ...,    0,    0,    0],
       ...,
       [  12, 2499,   82, ...,    0,    0,    0],
       [ 156,   12,  117, ...,    0,    0,    0],
       [  12,   18,  218, ...,    0,    0,    0]], dtype=int64)

In [11]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 130,977
Trainable params: 130,977
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

Epoch 1/10
2500/2500 [==============================]7 - accuracy: 0.30 - 1s 628ms/step - loss: 0.6950 - accuracy: 0.450 - 1s 422ms/step - loss: 0.6941 - accuracy: 0.500 - 1s 319ms/step - loss: 0.6945 - accuracy: 0.475 - 1s 257ms/step - loss: 0.6939 - accuracy: 0.500 - 1s 215ms/step - loss: 0.6941 - accuracy: 0.466 - 1s 186ms/step - loss: 0.6939 - accuracy: 0.485 - 1s 164ms/step - loss: 0.6939 - accuracy: 0.475 - 1s 146ms/step - loss: 0.6943 - accuracy: 0.444 - 1s 133ms/step - loss: 0.6940 - accuracy: 0.460 - 1s 121ms/step - loss: 0.6940 - accuracy: 0.454 - 1s 112ms/step - loss: 0.6938 - accuracy: 0.475 - 1s 104ms/step - loss: 0.6938 - accuracy: 0.469 - 1s 97ms/step - loss: 0.6938 - accuracy: 0.464 - 1s 91ms/step - loss: 0.6937 - accuracy: 0.46 - 1s 86ms/step - loss: 0.6936 - accuracy: 0.46 - 1s 82ms/step - loss: 0.6936 - accuracy: 0.47 - 1s 77ms/step - loss: 0.6936 - accuracy: 0.47 - 1s 74ms/step - loss: 0.6935 - accuracy: 0.47 - 1s 71ms/step - loss: 0.6935 - accuracy: 0.47 - 1s 68ms/

In [13]:
import matplotlib.pyplot as plt

history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))
plt.show()

<Figure size 1200x900 with 1 Axes>

<Figure size 1200x900 with 1 Axes>

In [14]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(8185, 16)


In [15]:
encoder = info.features['text'].encoder

In [16]:
import io

encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

In [17]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')